In [ ]:
!pip install torch_geometric
!pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00


In [ ]:
from google.colab import files
files.upload()

#jupytext --to ipynb Comparative_Cheb_GCN.py
# Don't forget to upload model.pth as well
!unzip numpy_data.zip

Saving numpy_data.zip to numpy_data.zip
Saving model_123.pt to model_123.pt
Archive:  numpy_data.zip
  inflating: AdjacencyMatrix.npy     
  inflating: new_data.npy            
  inflating: new_labels.npy          
  inflating: NodeIndex.npy           


In [ ]:

import time
start_time = time.time()

#
#!pip install torch_geometric
#!pip install pdf2image
#!apt-get install poppler-utils
## !pip install captum
#
#from google.colab import files
#files.upload()
#
#jupytext --to ipynb Comparative_Cheb_GCN.py
# Don't forget to upload model.pth as well
#!unzip numpy_data.zip

from torch_geometric.data import Data, DataLoader
from torch_geometric.datasets import TUDataset, Planetoid
from torch_geometric.nn import GCNConv, Set2Set
from torch_geometric.explain import GNNExplainer
import torch_geometric.transforms as T
import torch
#import torch.nn.functional as F
import os
from tqdm import tqdm, trange
#
import matplotlib.pyplot as plt

#Loading saved model architecture
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import ChebConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import TransformerConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn.pool import global_max_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        # self.conv1 = GCNConv(6, 512, improved = True)
        # self.conv2 = GCNConv(512, 256, improved = True)

        # self.conv1 = ChebConv(6, 512, K = 2)
        # self.conv2 = ChebConv(512,256, K = 2)

        self.conv1 = ChebConv(6, 256, K = 3)
        self.conv2 = ChebConv(256,256, K = 4)
        self.conv3 = ChebConv(256, 256, K = 5)
        # self.fc1 = Linear(256, 512)
        # self.fc2 = Linear(512, 256)

        # self.conv1 = GATConv(6, 256, heads = 4)
        # self.conv2 = GATConv(256*4,64, heads = 1, concat=False)

        # self.conv1 = TransformerConv(6, 200, heads = 3)
        # self.conv2 = TransformerConv(200*3,256, heads = 1, concat=False)

        self.lin = Linear(256, 120)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()

        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        # = global_max_pool(x, batch)  # [batch_size, hidden_channels]

        # x = self.fc1(x)
        # x = x.relu()
        # x = self.fc2(x)
        # x = x.relu()

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.2, training=self.training) #0.5
        x = self.lin(x)

        return x

model = GCN(hidden_channels=464)
print(model)

# print(os.getcwd()+ "/model.pth")
# model_save_location = os.getcwd()+ "/model.pth"
print(os.getcwd()+ "/model_123.pt")
model_save_location = os.getcwd()+ "/model_123.pt"
model.load_state_dict(torch.load(model_save_location, map_location=torch.device('cpu')))
# model.load_state_dict(torch.load(model_save_location))
print(model.conv3)
print(model.state_dict())

import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import os
import torch
import networkx as nx
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
print(os.getcwd())

# +
import time
#Extract Training Data
#Loads IEEE34 Bus Simulation Data

path = os.getcwd()

numpy_data = np.load(path+"/new_data.npy",allow_pickle = True)
numpy_data = np.squeeze(numpy_data)
print(numpy_data)
print(numpy_data.shape)
print(numpy_data.size)

value_data = np.load(path+"/new_labels.npy",allow_pickle = True)
print(value_data)
print(value_data.shape)
print(value_data.size)


NodeIndex = np.load(path+"/NodeIndex.npy",allow_pickle = True)
print(NodeIndex)
print(NodeIndex.shape)
print(NodeIndex.size)

AdjacencyMatrix = np.load(path+"/AdjacencyMatrix.npy",allow_pickle = True)
print(AdjacencyMatrix)
print(AdjacencyMatrix.shape)
print(AdjacencyMatrix.size)

# +
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import NormalizeFeatures
from torch.nn.functional import normalize

# encoder = ['SourceBus', '800', '802', '806', '808', '810', '812', '814', '814r', '850',
# '816', '818', '824', '820', '822', '826', '828', '830', '854', '832', '858',
# '834', '860', '842', '836', '840', '862', '844', '846', '848', '852r', '888', '856', '852', '864', '838', '890']
# print(encoder)
# print(len(encoder))
# research_paper_decoder = [0,0,1,2,3,4,5,6,6,6,6,7,8,9,10,11,8,12,12,13,14,15,21,15,16,16,16,17,18,18,13,13,19,13,20,22,23]

encoder = ['150', '150R', '149', '1', '2', '3', '7', '4', '5', '6', '8', '12',
'9', '13', '9R', '14', '34', '18', '11', '10', '15', '16', '17', '19',
'21', '20', '22', '23', '24', '25', '25R', '26', '28', '27', '31',
'33', '29', '30', '250', '32', '35', '36', '40', '37', '38', '39',
'41', '42', '43', '44', '45', '47', '46', '48', '49', '50', '51',
'151', '52', '53', '54', '55', '57', '56', '58', '60', '59', '61',
'62', '63', '64', '65', '66', '67', '68', '72', '97', '69', '70',
'71', '73', '76', '74', '75', '77', '86', '78', '79', '80', '81',
'82', '84', '83', '85', '87', '88', '89', '90', '91', '92', '93',
'94', '95', '96', '98', '99', '100', '450', '197', '101', '102',
'105', '103', '104', '106', '108', '107', '109', '300', '110',
'111', '112', '113', '114', '135', '152', '160R', '160', '61S', '610']

print(encoder)
print(len(encoder))
research_paper_decoder = [0,0,0,1,2,3,4,5,6,7,8,9,10,11,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,72,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,14,11,61,61,63,119]


FaultLocationLabels = value_data[:,3]

for n in range(len(FaultLocationLabels)):
    FaultLocationLabels[n]=research_paper_decoder[encoder.index(str(FaultLocationLabels[n]).upper())]


y = FaultLocationLabels.astype("int64")
y = torch.from_numpy(y)
x = numpy_data.astype("float32")
x = torch.from_numpy(x)

NodeIndex = NodeIndex.astype("int64")
NodeIndex = NodeIndex.T
NodeIndex = torch.from_numpy(NodeIndex)

print(y)
print(y.dtype)
print(x)
print(x.dtype)
print(NodeIndex)
print(NodeIndex.dtype)
#
print(x[0])
print(normalize(x[0]))
#
result_translator = np.unique(FaultLocationLabels.astype("int64")).tolist()
print()
total_data_list = []
for n in range(len(x)):
    #print(x[n])
    #print(y[n])
    DataObject = Data(x = x[n], edge_index = NodeIndex, y = y[n], is_undirected = True) #Testing with non-normalized data
    #DataObject = Data(x = x[n], edge_index = NodeIndex, y = y[n], is_undirected = True)
    DataObject.is_undirected = True
    total_data_list.append(DataObject)
#print('Y'*888)
#print(total_data_list[0].x)

print()
#print(f'Dataset: {total_data_list}:')
print('===================')
print(f'Number of graphs: {len(total_data_list)}')
print(f'Number of features: {total_data_list[0].num_features}')
#print(f'Number of classes: {total_data_list[0].num_classes}')

data = total_data_list[0]  # Get the first graph object.
#print(data)
#print(data.y)

print()
print(NormalizeFeatures(data.x))
print(data.x)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

from random import shuffle
torch.manual_seed(12345)
#total_data_list = total_data_list.shuffle()
shuffle(total_data_list)

train_dataset = total_data_list[:26000] #9150 is half
test_dataset = total_data_list[26000:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')


print(test_dataset[0].x)
print(test_dataset[0])
#print(train_dataset[0].x)

for sample in range(len(test_dataset)):
    # noise = np.random.normal(1,0.03, size = (37,6)) #0.09 #Uncomment for noise
    # noise = noise.astype("float32")
    # noise = torch.from_numpy(noise)
    # test_dataset[sample].x = noise*test_dataset[sample].x
    test_dataset[sample].x = normalize(test_dataset[sample].x)
    #print(sample)

print('Y'*888)
#print(train_dataset[0].x)
print(test_dataset[0].x)
print(test_dataset[0])
##noise = np.random.normal(1,0.09, size = (16,6))
##noise = noise.astype("float32")
##noise = torch.from_numpy(noise)
print('=============================================================')
##
print(train_dataset[0].x)
print(train_dataset[0])
#print(train_dataset[0].x)

for sample in range(len(train_dataset)):
#    noise = np.random.normal(1,0.09, size = (16,6))
#    #    noise = np.random.normal(1,0.09, size = (1,96))
#    noise = noise.astype("float32")
#    noise = torch.from_numpy(noise)
##    print(noise)
##    X_test[sample] = X_test[sample]*noise[0]
#    train_dataset[sample].x = noise*train_dataset[sample].x
    train_dataset[sample].x = normalize(train_dataset[sample].x)
    #print(sample)

print('Y'*888)
#print(train_dataset[0].x)
print(train_dataset[0].x)
print(train_dataset[0])
##print(noise*test_dataset[0].x)
# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

print("$"*200)

#################################################################################
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch_geometric.transforms as T
from torch_geometric.datasets import ExplainerDataset
from torch_geometric.datasets.graph_generator import BAGraph
from torch_geometric.explain import Explainer, CaptumExplainer
from torch_geometric.explain import GNNExplainer, GraphMaskExplainer
from torch_geometric.explain import DummyExplainer, PGExplainer

from torch_geometric.nn import GCN
from torch_geometric.utils import k_hop_subgraph
from typing import Tuple

import torch
from torch import Tensor

from torch_geometric.explain import Explainer, Explanation
from torch_geometric.explain.config import ExplanationType, ModelMode

pos_fidelity_list = []
neg_fidelity_list = []
characterization_list = []
# topk_list = [1,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80]
topk_list = [10]
for topk in topk_list:
#     explainer2 = Explainer(
#         model=model,
#     #    algorithm=GNNExplainer(epochs=200),
#     #    algorithm=PGExplainer(epochs=2),
#         algorithm=GraphMaskExplainer(num_layers = 3, epochs=200),
#     #    algorithm=DummyExplainer(),
#         explanation_type='model',
#         node_mask_type='attributes',
#         edge_mask_type='object',
#         model_config=dict(
#             mode='multiclass_classification',
#             task_level='graph',
#             return_type='log_probs',
#         ),
#         threshold_config=dict(
# #            threshold_type='topk',
#             threshold_type='topk_hard',
#             value =topk,
#         ),
#     )

    explainer = Explainer(
        model=model,
        # algorithm=CaptumExplainer(attribution_method = "IntegratedGradients"),
      #  algorithm=CaptumExplainer(attribution_method = "Saliency"),
      #  algorithm=CaptumExplainer(attribution_method = "Deconvolution"),
      #  algorithm=CaptumExplainer(attribution_method = "GuidedBackprop"),
    #    algorithm=CaptumExplainer(attribution_method = "ShapleyValueSampling"),
       algorithm=CaptumExplainer(attribution_method = "InputXGradient"),
        explanation_type='model',
     #   explanation_type='phenomenon',
        node_mask_type='attributes',
        edge_mask_type='object',
        model_config=dict(
            mode='multiclass_classification',
            task_level='graph',
            return_type='log_probs',
        ),
        threshold_config=dict(
            # threshold_type='topk',
            threshold_type='topk_hard',
            value =topk,
        ),
    )
    #print(explainer.algorithm.attribution_method_class)
    count = 0
    explain_y_hat_list = []
    complement_y_hat_list = []
    y_list = []
    y_hat_list = []

    for batch in test_loader:
        count+=1
        Edge_Index = batch.edge_index[:,:252]
        Batch = batch.batch[:130]
        label1 = batch.y[0].numpy().tolist()#
        input = batch.x[0:130]#
        explanation = explainer(x = input, edge_index = Edge_Index, batch = Batch)#, target = label1
        y_hat = model(input, Edge_Index, Batch).argmax(dim=1).item()
    #path = 'subgraph.pdf'
    #explanation.visualize_graph(path)
        node_mask = explanation.get('node_mask')
        edge_mask = explanation.get('edge_mask')
        kwargs = {key: explanation[key] for key in explanation._model_args}
    #    y = explanation.target
        y = label1
        print(node_mask)
        print(edge_mask)
        print(node_mask.size())
        print(edge_mask.size())
    #    print(node_mask.dtype)
    #    print(edge_mask.dtype)
    #tensor_double = torch.tensor([1.0, 2.0, 3.0], dtype=torch.float64)
        node_mask = torch.tensor(node_mask, dtype=torch.float32)#.detach().numpy().astype('double')
        edge_mask = torch.tensor(edge_mask, dtype=torch.float32)#.detach().numpy().astype('double')

        #Two lines below focus on converting the soft masks to hard masks
    #    node_mask = torch.tensor((node_mask-node_mask.mean())>0, dtype=torch.float32)
    #    edge_mask = torch.tensor((edge_mask-edge_mask.mean())>0, dtype=torch.float32)

        print(node_mask)
    #    print(1 - node_mask)
        print(edge_mask)

        explain_y_hat = explainer.get_masked_prediction(
                explanation.x,
                explanation.edge_index,
                node_mask,
                edge_mask,
                **kwargs,
            )
        explain_y_hat = explainer.get_target(explain_y_hat)

        complement_y_hat = explainer.get_masked_prediction(
            explanation.x,
            explanation.edge_index,
            1. - node_mask if node_mask is not None else None,
            1. - edge_mask if edge_mask is not None else None,
            **kwargs,
        )
        complement_y_hat = explainer.get_target(complement_y_hat)

        explain_y_hat_list.append(explain_y_hat.numpy()[0])
        complement_y_hat_list.append(complement_y_hat.numpy()[0])
        y_list.append(y)
        y_hat_list.append(y_hat)

        print(count)
        print(explain_y_hat.numpy()[0])
        print(complement_y_hat.numpy()[0])
        print(y)
        print(y_hat)
        print("#########")

        # if count > 2:
          # break

    explain_y_hat_list = np.array(explain_y_hat_list)
    complement_y_hat_list = np.array(complement_y_hat_list)
    y_list = np.array(y_list)
    y_hat_list = np.array(y_hat_list)

    print(explain_y_hat_list )
    print(complement_y_hat_list )
    print(y_list )
    print(y_hat_list)

    pos_fidelity = 1. - (complement_y_hat_list == y_hat_list).astype('float').mean()
    neg_fidelity = 1. - (explain_y_hat_list == y_hat_list).astype('float').mean()

    print("Positive Fidelity = ", pos_fidelity) #positive fidelity reflects necessary explanation
    # It's better for positive fidelity to be 1
    print("Negative Fidelity = ", neg_fidelity) #negative fidelity reflects sufficient explanation
    # It's better for negative fidelity to be 0
    CharacterizationScore = 1./((0.5/pos_fidelity)+(0.5/(1-neg_fidelity)))
    print(CharacterizationScore)
    #print("Fidelity Curve AUC = ",  pos_fidelity/(1 - neg_fidelity))

    pos_fidelity_list.append(pos_fidelity)
    neg_fidelity_list.append(neg_fidelity)
    characterization_list.append(CharacterizationScore)

print(topk_list)
print(pos_fidelity_list)
print(neg_fidelity_list)
print(characterization_list)

Topk_Results = pd.DataFrame(data = {'TopK' : topk_list,
    'Fidelity+' : pos_fidelity_list,
    'Fidelity-' : neg_fidelity_list,
    'Characterization' : characterization_list})

#Topk_Results.to_csv(path+'/'+explainer.algorithm.attribution_method_class.__name__+'_topk_results.csv', index = False)
Topk_Results.to_csv(path+'/'+explainer.algorithm.attribution_method_class.__name__+'_topk_hard_results.csv', index = False)

print("  "+explainer.algorithm.attribution_method_class.__name__)
#print(type(explainer.algorithm.attribution_method_class))
print(path+'/'+explainer.algorithm.attribution_method_class.__name__+'_topk_results.csv')
print(Topk_Results)

end_time = time.time()
print("Run Time = ", end_time-start_time)



# from IPython.display import IFrame



# # Specify the path to your PDF file
# pdf_path = 'subgraph.pdf'

# # Display the PDF inline
# IFrame(pdf_path, width=600, height=300)

# from pdf2image import convert_from_path

# images = convert_from_path("subgraph.pdf")
# images[0]  # first page




#
#from pdf2image import convert_from_path
#from IPython.display import display, Image
#
#images = convert_from_path("subgraph.pdf")
#for i, image in enumerate(images):
#    fname = "image" + str(i) + ".png"
#    image.save(fname, "PNG")
#Image(fname, width=800, height=1500)
#
#print(NodeIndex)
#print(NodeIndex.shape)
#print(NodeIndex.size)

#tensor([[1.6330e-01, 1.6028e+01, 5.6377e+00, 3.7685e+01, 1.3250e+00, 7.6285e+00],
#        [1.9494e+00, 1.8429e+01, 5.0191e+00, 3.7182e+01, 6.1602e-02, 6.6159e+00],
#        [1.7525e+00, 1.3168e+01, 3.6308e+00, 3.8188e+01, 3.1286e-01, 1.1010e+01],
#        [1.9209e+00, 2.4399e+00, 1.3984e+00, 2.8718e+01, 3.1227e+00, 4.9003e+00],
#        [4.8037e+00, 7.0450e+00, 4.3841e+00, 3.4673e+01, 4.6275e+00, 3.2360e+00],
#        [2.6122e+00, 2.1249e+00, 5.0562e+00, 1.1951e+00, 3.9602e+00, 2.7298e+00],
#        [1.1510e+01, 1.8526e+01, 1.3589e+01, 2.3464e+01, 1.6671e+01, 1.9874e+01],
#        [1.0703e+01, 1.7369e+01, 1.4182e+01, 7.4979e+00, 1.4240e+01, 2.6974e+01],
#        [4.9560e+00, 8.2325e+00, 6.4807e+00, 2.5753e+01, 5.8989e+00, 2.3319e+01],
#        [2.1680e+00, 3.1957e-01, 1.2022e+00, 4.9803e+01, 2.4505e+00, 1.2831e+01],
#        [3.6352e+00, 2.6156e+00, 7.2151e+00, 4.4443e+01, 4.8684e+00, 7.7395e+00],
#        [5.1270e-01, 5.6600e-01, 1.5779e+01, 2.1507e+01, 5.9178e+00, 2.3970e+00],
#        [5.9067e-01, 1.2074e+00, 1.1073e+01, 2.5636e+01, 3.0617e+00, 6.8622e+00],
#        [3.3402e-01, 3.6277e-01, 2.2604e+01, 6.4051e-01, 7.8227e+00, 1.8952e+00],
#        [3.0360e-04, 4.2593e-01, 1.7118e+01, 4.0074e+00, 8.8157e+00, 2.9874e+00],
#        [1.0799e+00, 2.9640e+00, 1.0315e+01, 1.1820e+01, 4.3091e+00, 2.4597e+00],
#        [7.0108e-01, 3.6368e-01, 8.3599e+00, 1.1408e+01, 1.1791e+00, 4.3466e+00],
#        [1.0583e+00, 6.5089e-01, 8.4722e+00, 6.4298e+00, 1.2448e+00, 2.9624e+00],
#        [8.7381e-01, 2.3920e+00, 1.3151e+01, 3.1015e+00, 1.5988e+00, 1.8356e+00],
#        [1.3442e+00, 2.4827e+00, 1.3564e+01, 2.3057e+01, 1.2681e+00, 3.8963e+00],
#        [1.2420e+00, 3.6691e+00, 1.8933e+01, 5.8242e+01, 8.1786e-01, 1.0273e+01],
#        [1.6988e+00, 2.8533e+00, 1.8349e+01, 4.0507e+01, 8.5044e-01, 7.8924e+00],
#        [1.2453e+00, 2.6515e+00, 1.5632e+01, 2.1877e+01, 1.5727e+00, 3.7594e+00],
#        [1.4627e+00, 3.3610e+00, 1.3347e+01, 2.6755e+01, 1.5356e+00, 5.6953e+00],
#        [1.1680e+00, 1.9932e+00, 1.8786e+01, 7.9231e+00, 4.9314e-01, 8.5603e-01],
#        [4.8731e-01, 9.2746e-01, 1.1566e+01, 4.0931e+00, 8.6125e-01, 8.3077e-01],
#        [7.5199e-01, 1.3115e+00, 1.5971e+01, 9.5758e+00, 1.0225e+00, 5.8268e-01],
#        [1.4688e+00, 3.1322e+00, 1.0954e+01, 6.6173e+00, 1.5992e+00, 2.6018e+00],
#        [1.2650e+00, 3.1655e+00, 9.0339e+00, 7.6423e+00, 1.7518e+00, 4.6447e-01],
#        [9.5454e-01, 2.1755e+00, 5.7167e+00, 1.3194e+01, 1.5726e+00, 6.9864e-01],
#        [1.5696e+00, 2.5563e+00, 1.0295e+01, 8.5099e+00, 1.9298e+00, 1.3850e+00],
#        [8.6243e-01, 1.9521e+00, 8.8169e+00, 6.0340e+00, 1.2954e+00, 9.3387e-02],
#        [1.2535e+00, 4.8155e+00, 1.0605e+01, 3.1666e+00, 4.0645e+00, 3.1496e-01],
#        [8.9946e-01, 1.7818e+00, 9.7507e+00, 4.4109e-01, 1.1955e+00, 4.0798e-01],
#        [1.5681e+00, 4.5888e+00, 1.3767e+01, 3.6835e+01, 3.9696e+00, 8.2341e+00],
#        [3.7807e-01, 7.0457e-01, 1.1269e+01, 4.2274e+00, 4.1955e-01, 7.8272e-01],
#        [5.3371e-01, 9.9733e-01, 5.4828e+00, 1.6085e+00, 9.7913e-01, 1.4172e+00]])
#tensor([ 0.6331,  0.9813,  0.5346,  1.4886,  2.6457,  7.9937, 13.1034, 11.9424,
#         2.1656,  2.3123,  4.0676,  5.5632,  1.0517,  3.5155,  2.7599,  2.2101,
#         1.4171,  0.8244,  0.2534,  0.1072,  0.3916,  0.0743,  0.4779,  0.6228,
#         2.3477,  0.6322,  0.9771,  0.7180,  2.0766,  0.8709,  0.3458,  0.5010,
#         0.8927,  1.5414, 17.0340,  0.6270,  2.7655,  0.2935,  0.4884,  0.2197,
#         6.0778,  0.6790, 11.2975, 15.0941,  3.1437,  0.4076,  4.6496,  5.7883,
#         3.0723,  1.7596,  1.7273,  1.8248,  0.7651,  0.9515,  1.5299,  0.2637,
#         0.6066,  0.8976,  0.3030,  0.2072,  4.5935,  1.1595,  2.0674,  2.7745,
#         0.5663,  0.8655,  1.3210,  0.2029,  1.4588,  2.2045, 19.3664,  0.4314])


GCN(
  (conv1): ChebConv(6, 256, K=3, normalization=sym)
  (conv2): ChebConv(256, 256, K=4, normalization=sym)
  (conv3): ChebConv(256, 256, K=5, normalization=sym)
  (lin): Linear(in_features=256, out_features=120, bias=True)
)
/content/model_123.pt
ChebConv(256, 256, K=5, normalization=sym)
OrderedDict([('conv1.bias', tensor([ 0.0560, -0.0923,  0.0211,  0.1891, -0.3570, -0.0243, -0.0082, -0.0498,
        -0.2476, -0.2073, -0.1551,  0.0744,  0.0436, -0.0038, -0.1946, -0.2249,
         0.1485,  0.0596, -0.1403, -0.0865, -0.0554, -0.0322, -0.1644, -0.2891,
        -0.0723, -0.0622,  0.1296, -0.3368, -0.0972,  0.0546, -0.4115, -0.0499,
         0.0018,  0.0772, -0.0011, -0.3186, -0.0530, -0.0161,  0.1959,  0.0350,
        -0.1036, -0.3501,  0.0229, -0.0258, -0.1350, -0.0788,  0.0172,  0.1075,
         0.1067,  0.0942,  0.0376,  0.2836, -0.0751, -0.0666,  0.1361, -0.1322,
        -0.0099, -0.0077,  0.1709, -0.1765, -0.3071, -0.5233,  0.0284,  0.0188,
        -0.1241,  0.0721,  0.0533, -0.

<ipython-input-4-dfefc4f41637>:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_location, map_location=torch.device('cpu')))


[[[2398.8 0.0 2398.1 -120.0 2398.51 120.0]
  [2157.99 -0.1 2157.14 -120.1 2157.64 120.0]
  [2157.96 -0.1 2157.13 -120.1 2157.63 120.0]
  ...
  [3.90842 -58.7 4.51612 174.1 3.8953 50.7]
  [3.90842 -58.7 4.51612 174.1 3.8953 50.7]
  [0.455533 -59.0 0.520818 174.7 0.445051 50.3]]

 [[2398.8 0.0 2398.1 -120.0 2398.51 120.0]
  [2157.99 -0.1 2157.14 -120.1 2157.64 120.0]
  [2157.96 -0.1 2157.13 -120.1 2157.63 120.0]
  ...
  [3.85896 -59.2 4.48408 173.8 3.85716 50.4]
  [3.85896 -59.2 4.48408 173.8 3.85717 50.4]
  [0.450359 -59.4 0.51607 174.4 0.441351 49.9]]

 [[2398.8 0.0 2398.1 -120.0 2398.51 120.0]
  [2157.99 -0.1 2157.14 -120.1 2157.64 120.0]
  [2157.96 -0.1 2157.13 -120.1 2157.63 120.0]
  ...
  [3.81044 -59.6 4.45216 173.5 3.81996 50.0]
  [3.81044 -59.6 4.45216 173.5 3.81996 50.0]
  [0.445281 -59.8 0.511402 174.1 0.437689 49.4]]

 ...

 [[2401.4 0.0 2401.39 -120.0 2401.36 120.0]
  [2641.39 0.0 2641.38 -120.0 2641.34 120.0]
  [2641.39 0.0 2641.38 -120.0 2641.34 120.0]
  ...
  [1048.31 4.8

/usr/local/lib/python3.10/dist-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
<ipython-input-4-dfefc4f41637>:404: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  node_mask = torch.tensor(node_mask, dtype=torch.float32)#.detach().numpy().astype('double')
<ipython-input-4-dfefc4f41637>:405: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_mask = torch.tensor(edge_mask, dtype=torch.float32)#.detach().numpy().astype('double')


Streaming output truncated to the last 5000 lines.
        [0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 1.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.